In [82]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from scipy import stats
import seaborn as sns

In [83]:
file_path = r"C:\Python\HessiSolarFlare2018.csv"  #2002to2018.csv
df = pd.read_csv(file_path)
formatT = '%Y-%m-%d %H:%M:%S'
df['Datetime'] = pd.to_datetime(df['start.date'] + ' ' + df['start.time'], format=formatT)
df = df.set_index(pd.DatetimeIndex(df['Datetime']))
df = df.drop(columns=['start.date', 'start.time', 'Datetime'])
#organizo en otros dataframes las columnas de duracion.s y energia.kev en orden descendente
Emin = []
Emax = []
Eprom = []
for i in df['energy.kev']:
    result = re.findall(r'[\d\.\d]+', i)
    Emax.append(float(result[1]))
    Emin.append(float(result[0]))
    Eprom.append((float(result[0]) + float(result[1]))/2)
df['EnergiaMin']=Emin
df['EnergiaMax']=Emax
df['EnergiaProm']=Eprom
#saca las flags en distintos dataframes
df1=pd.DataFrame(df.value_counts('flag.1'),columns=['Bandera1']).reset_index()
df2=pd.DataFrame(df.value_counts('flag.2'),columns=['Bandera2']).reset_index()
df3=pd.DataFrame(df.value_counts('flag.3'),columns=['Bandera3']).reset_index()
df4=pd.DataFrame(df.value_counts('flag.4'),columns=['Bandera4']).reset_index()
df5=pd.DataFrame(df.value_counts('flag.5'),columns=['Bandera5']).reset_index()
#Filtro para datos de calidad Q1 (la mayor posible de la base de datos con respecto a la flag Qn)
Q13=df['flag.3']=="Q1"
Q14=df['flag.4']=="Q1"
Q15=df['flag.5']=="Q1"
dfQ13=df[Q13]
dfQ1=pd.concat([df[Q13],df[Q14],df[Q15]]).sort_values('Datetime')
dfQ1.value_counts('flag.3')
print(dfQ1)

                        flare      peak       end  duration.s  peak.c/s  \
Datetime                                                                  
2002-02-12 21:44:08   2021228  21:45:06  21:48:56         288         7   
2002-02-13 23:28:32   2021329  23:30:38  23:31:56         204        18   
2002-02-13 23:31:56   2021355  23:34:06  23:52:56        1260        56   
2002-02-14 22:05:20   2021463  22:14:14  22:19:28         848        27   
2002-02-17 10:30:20   2021726  10:34:14  10:34:24         244        36   
...                       ...       ...       ...         ...       ...   
2017-11-17 00:23:48  17111701  00:28:10  00:30:32         404       304   
2018-01-18 07:29:24  18011801  07:34:58  07:42:56         812       256   
2018-01-20 04:50:48  18012001  04:51:58  04:53:28         160        20   
2018-01-20 04:53:28  18012002  04:54:30  04:57:28         240        16   
2018-03-03 04:04:08  18030301  04:06:22  04:12:20         492        42   

                     tot

In [81]:
#graficas y analisis de datos
"""hipotesis estadistica de eventos solares con la bandera de posicion valida P1
   con datos de calidad Q1"""
P13=dfQ1['flag.3']=="P1"
P14=dfQ1['flag.4']=="P1"
P15=dfQ1['flag.5']=="P1"
#dataframe con los eventos solares observados (validos y posibles con calidad Q1)
dfsf=pd.concat([dfQ1[P13],dfQ1[P14],dfQ1[P15]]).sort_index()
#se extraen datos nulos
dfsffull=dfsf.dropna(how='any')
#imprimo la grafica
#sns.barplot(data=dfsffull,x=dfsffull['duration.s'],y='EnergiaMin')
#promedios
PromEnergiaP=dfsffull['EnergiaProm'].mean()
PromEnergiaMin=dfsffull['EnergiaMin'].mean()
PromEnergiaMax=dfsffull['EnergiaMax'].mean()
Promduracion=dfsffull['duration.s'].mean()
print("el promedio de la energia promedio es ",PromEnergiaP, "\n el primedio de la energia min es",PromEnergiaMin,"\n el promedio de la energia maxima es",PromEnergiaMax,"\n promedio de duracion",Promduracion)
dfsffull
#sns.displot(data=df, x='')

el promedio de la energia promedio es  21.645833333333332 
 el primedio de la energia min es 14.106884057971014 
 el promedio de la energia maxima es 29.184782608695652 
 promedio de duracion 944.2608695652174


,flare,peak,end,duration.s,peak.c/s,total.counts,energy.kev,x.pos.asec,y.pos.asec,radial,active.region.ar,flag.1,flag.2,flag.3,flag.4,flag.5,EnergiaMin,EnergiaMax,EnergiaProm
Datetime,,,,,,,,,,,,,,,,,,,
2002-02-18 20:18:40,2021825,20:53:30,21:12:48,3248,304,2736546.0,12-25,-272,-202,339,9830,a0,A1,P1,PE,Q1,12.0,25.0,18.5
2002-02-18 22:30:12,2021886,22:30:18,22:45:16,904,18,68890.0,6-12,-269,-235,357,9830,a0,A1,P1,PE,Q1,6.0,12.0,9.0
2002-02-26 21:50:04,2022648,21:51:30,22:02:04,720,17,34426.0,6-12,800,-260,841,9839,A0,a1,P1,PE,Q1,6.0,12.0,9.0
2002-03-01 17:07:56,2030126,17:12:50,17:19:20,684,208,235168.0,12-25,920,-334,979,0,a0,A1,P1,PE,Q1,12.0,25.0,18.5
2002-03-14 01:35:32,2031402,01:46:26,01:55:44,1212,7408,23628974.0,50-100,-377,-76,385,0,a0,A1,EE,P1,Q1,50.0,100.0,75.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016-09-22 18:41:12,16092226,18:41:50,18:43:52,160,24,4034.0,6-12,941,186,959,0,A0,a1,P1,PE,Q1,6.0,12.0,9.0
2016-10-17 00:27:24,16101702,00:34:10,00:48:24,1260,80,115703.0,12-25,973,-241,1003,2599,a0,A1,P1,PE,Q1,12.0,25.0,18.5
2016-11-29 07:05:12,16112908,07:09:58,07:21:08,956,464,123832.0,100-300,-849,-145,862,2615,a0,A1,P1,PE,Q1,100.0,300.0,200.0
